In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion

In [42]:
mainData = pd.read_csv('game_logs.csv')
df = mainData.copy()

C:\Users\amjad\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (12,13,14,15,19,20,81,83,85,87,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [43]:
mainData.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,5,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,6,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,7,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,2,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,3,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [44]:
mainData.shape

(171907, 161)

In [45]:
df.dtypes

date                    int64
number_of_game          int64
day_of_week             int64
v_name                 object
v_league               object
                       ...   
h_player_9_id          object
h_player_9_name        object
h_player_9_def_pos    float64
additional_info        object
acquisition_info       object
Length: 161, dtype: object

In [46]:
# change day_of_week type into daytime
df['day_of_week'] = df['day_of_week'].astype('int')

In [47]:
#Delete repeated columns
df = df[df.columns.drop(list(df.filter(regex='name')))]

In [48]:
df.shape

(171907, 127)

- Deleted all the columns that has "name" in the data which are 34, because there are alwayes a columne with the "name id" after it.
- There is "v_name", "h_name" and I don't think it is as the same as "h_league" and "v_league" so I don't know if I should return it back! 
- The date column is unusual and I don't know how to deal with it, should I change it to datetime!
- Change "day_of_week" type into datetime

## Feature Engeneering

* Encoded all the players from different columns so they have the same number
* I don't know what are these columns and how should I encode them (completion, h_line_score, v_line_score)

In [49]:
# Encoded all the players so they have the same number even if they are in different columns
# I should create a function instead of all these lines but I don't know how =)

country_map = {country:i for i, country in enumerate(df.stack().unique())}
df['h_player_9_id'] = df['h_player_9_id'].map(country_map)    
df['h_player_8_id'] = df['h_player_8_id'].map(country_map)
df['h_player_7_id'] = df['h_player_7_id'].map(country_map)
df['h_player_6_id'] = df['h_player_6_id'].map(country_map)
df['h_player_5_id'] = df['h_player_5_id'].map(country_map)
df['h_player_4_id'] = df['h_player_4_id'].map(country_map)
df['h_player_3_id'] = df['h_player_3_id'].map(country_map)
df['h_player_2_id'] = df['h_player_2_id'].map(country_map)
df['h_player_1_id'] = df['h_player_1_id'].map(country_map)
df['v_player_9_id'] = df['v_player_9_id'].map(country_map)
df['v_player_8_id'] = df['v_player_8_id'].map(country_map)
df['v_player_7_id'] = df['v_player_7_id'].map(country_map)
df['v_player_6_id'] = df['v_player_6_id'].map(country_map)
df['v_player_5_id'] = df['v_player_5_id'].map(country_map)
df['v_player_4_id'] = df['v_player_4_id'].map(country_map)
df['v_player_3_id'] = df['v_player_3_id'].map(country_map)
df['v_player_2_id'] = df['v_player_2_id'].map(country_map)
df['v_player_1_id'] = df['v_player_1_id'].map(country_map)
df['h_starting_pitcher_id'] = df['h_starting_pitcher_id'].map(country_map)
df['v_starting_pitcher_id'] = df['v_starting_pitcher_id'].map(country_map)
df['winning_rbi_batter_id'] = df['winning_rbi_batter_id'].map(country_map)
df['saving_pitcher_id'] = df['saving_pitcher_id'].map(country_map)
df['losing_pitcher_id'] = df['losing_pitcher_id'].map(country_map)
df['winning_pitcher_id'] = df['winning_pitcher_id'].map(country_map)
df['h_manager_id'] = df['h_manager_id'].map(country_map)
df['v_manager_id'] = df['v_manager_id'].map(country_map)
df['rf_umpire_id'] = df['rf_umpire_id'].map(country_map)
df['lf_umpire_id'] = df['lf_umpire_id'].map(country_map)
df['3b_umpire_id'] = df['3b_umpire_id'].map(country_map)
df['2b_umpire_id'] = df['2b_umpire_id'].map(country_map)
df['1b_umpire_id'] = df['1b_umpire_id'].map(country_map)
df['hp_umpire_id'] = df['hp_umpire_id'].map(country_map)
df['h_league'] = df['h_league'].map(country_map)
df['v_league'] = df['v_league'].map(country_map)

In [50]:
#functions that make days as sycle 

def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

In [9]:
# encode special columns that don't have any value connected with other columns
# unfotinatly it gave an error that so many data to unpack so I will sperate them
extraction = ColumnTransformer([
    ('lbs', LabelEncoder(), ['acquisition_info'],["park_id"]),
    ('date', FeatureUnion([
        ('doy_cos', cos_transformer(365)),
        ('doy_sin', sin_transformer(365)),
    ]), ['date']),
    ('deptc', OneHotEncoder(), ['day_night'], ["forefeit"],["protest"]),
    ('day_of_week', FeatureUnion([
        ('dow_cos', cos_transformer(7)),
        ('dow_sin', sin_transformer(7)),
    ]), ['day_of_week']),
], remainder='passthrough')

df = extraction.fit_transform(df)

ValueError: too many values to unpack (expected 3)

In [51]:
lable = LabelEncoder()
hotco = OneHotEncoder()

In [52]:
# it gives error when I use 'OneHotEncoder' function coz it's not iterable so I just LabelEncoder all of them
#devided into muliple cells so it wont take long time

df['acquisition_info'] = lable.fit_transform(df['acquisition_info'])
df['park_id'] = lable.fit_transform(df['park_id'])

In [53]:
df['day_night'] = lable.fit_transform(df['day_night'])
df['forefeit'] = lable.fit_transform(df['forefeit'])

In [54]:
df['protest'] = lable.fit_transform(df['protest'])

In [59]:
df.to_csv("ML_wranlge.csv",index=False)